In [1]:
import pandas as pd
import plotly.express as px

# Load Gapminder dataset from Plotly
gapminder_df = px.data.gapminder()

# Display the first few rows of the dataset
gapminder_df.head()


,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4


In [2]:
# Scatter plot to visualize population over time for each country
fig = px.scatter(gapminder_df, x="year", y="pop", color="continent",
                 size="gdpPercap", hover_name="country", title="Population Growth Over Time")
fig.show()


In [3]:
from sklearn.model_selection import train_test_split

# Select features and target variable
X = gapminder_df[['year', 'gdpPercap', 'lifeExp']]
y = gapminder_df['pop']

# Split the data into training and test sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model performance
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model performance
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 1.928545949336164e+16
Mean Squared Error: 1.928545949336164e+16


In [6]:
pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 26.8 MB/s eta 0:00:00


In [7]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the dashboard
app.layout = html.Div(style={'backgroundColor': 'black', 'height': '100vh'}, children=[
    html.Div([
        html.H1("World Population Growth Dashboard", style={'textAlign': 'center', 'color': 'white'}),

        # Dropdown for selecting country
        html.Label('Select Country:', style={'color': 'white'}),
        dcc.Dropdown(
            id='country-dropdown',
            options=[{'label': country, 'value': country} for country in gapminder_df['country'].unique()] + [{'label': 'All Countries', 'value': 'all'}],
            value='all',
            style={'color': 'black'}
        ),

        # Slider for selecting year
        html.Label('Select Year:', style={'color': 'white'}),
        dcc.Slider(
            id='year-slider',
            min=gapminder_df['year'].min(),
            max=gapminder_df['year'].max(),
            value=gapminder_df['year'].max(),
            marks={str(year): str(year) for year in gapminder_df['year'].unique()},
            step=None
        ),
    ], style={'width': '80%', 'margin': '0 auto'}),

    # First Row: World Map
    html.Div([
        dcc.Graph(id='population-map', style={'height': '80vh'}),
    ], style={'margin': '20px 0'}),

    # Second Row: Population Trend and Future Prediction
    html.Div([
        # First Column: Population Trends Over Time
        html.Div([
            dcc.Graph(id='population-trend', style={'height': '45vh'}),
        ], style={'width': '48%', 'display': 'inline-block'}),

        # Second Column: Future Prediction
        html.Div([
            html.Label('Enter Year for Future Prediction:', style={'color': 'white'}),
            dcc.Input(id='predict-year', type='number', placeholder="Enter year", style={'color': 'black'}),
            html.Button('Predict Population', id='predict-button', n_clicks=0, style={'color': 'black'}),
            dcc.Graph(id='future-prediction', style={'height': '45vh'}),
        ], style={'width': '48%', 'display': 'inline-block', 'float': 'right'})
    ])
])

# Callback to update the population map based on year and country
@app.callback(
    Output('population-map', 'figure'),
    [Input('year-slider', 'value'),
     Input('country-dropdown', 'value')]
)
def update_map(selected_year, selected_country):
    if selected_country == 'all':
        filtered_df = gapminder_df[gapminder_df['year'] == selected_year]
    else:
        filtered_df = gapminder_df[(gapminder_df['year'] == selected_year) & (gapminder_df['country'] == selected_country)]

    fig = px.choropleth(
        filtered_df,
        locations="iso_alpha",
        color="pop",
        hover_name="country",
        title=f"Population in {selected_year}",
        color_continuous_scale='Viridis'
    )
    fig.update_layout(
        paper_bgcolor='black',
        plot_bgcolor='black',
        font_color='white'
    )
    return fig

# Callback to update the population trend based on selected country
@app.callback(
    Output('population-trend', 'figure'),
    [Input('country-dropdown', 'value')]
)
def update_trend(selected_country):
    if selected_country == 'all':
        trend_df = gapminder_df
    else:
        trend_df = gapminder_df[gapminder_df['country'] == selected_country]

    fig = px.line(
        trend_df,
        x='year', y='pop', color='continent',
        title=f"Population Trends for {selected_country if selected_country != 'all' else 'All Countries'}"
    )
    fig.update_layout(
        paper_bgcolor='black',
        plot_bgcolor='black',
        font_color='white'
    )
    return fig

# Callback for future population prediction
@app.callback(
    Output('future-prediction', 'figure'),
    [Input('predict-button', 'n_clicks')],
    [Input('predict-year', 'value'),
     Input('country-dropdown', 'value')]
)
def predict_future_population(n_clicks, predict_year, selected_country):
    if predict_year is None:
        return go.Figure()  # Return empty figure if no year is provided

    if selected_country == 'all':
        avg_gdp = gapminder_df['gdpPercap'].mean()
        avg_life_exp = gapminder_df['lifeExp'].mean()
        predicted_population = model.predict([[predict_year, avg_gdp, avg_life_exp]])
        fig_title = f"Predicted Global Population in {predict_year}"
    else:
        country_data = gapminder_df[gapminder_df['country'] == selected_country].mean()
        predicted_population = model.predict([[predict_year, country_data['gdpPercap'], country_data['lifeExp']]])
        fig_title = f"Predicted Population of {selected_country} in {predict_year}"

    fig = go.Figure(go.Indicator(
        mode="number",
        value=predicted_population[0],
        title={"text": fig_title}
    ))

    fig.update_layout(
        paper_bgcolor='black',
        font_color='white'
    )
    return fig

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>